# TFL Bike data Analysis
___

#### Exploratory data analysis as part of my MSc thesis, "Using machine learning to predict Transport for London bike sharing habits in the post COVID-19 era".

The following code for downloading the data has been adopted from [Markus Hauru's](https://github.com/mhauru) analysis, 'Predicting Boris Bike usage'.



In [10]:
# importing libraries

import os
import pickle
import requests
import zipfile
import pandas as pd
import numpy as np
import scipy as sp
import statsmodels.api as sm
from sklearn import linear_model, svm, neighbors, tree
from matplotlib import pyplot as plt
import matplotlib
import seaborn as sns
from pathlib import Path
from timeit import default_timer as timer
from IPython.display import set_matplotlib_formats
from urllib.parse import urlparse
import openpyxl

try:
    import xlrd
except Exception as e:
    msg = (
        "Please install the package xlrd: `pip install --user xlrd`"
        "It's an optional requirement for pandas, and we'll be needing it."
    )
    print(msg)
    raise e

In [11]:
# For pretty and exportable matplotlib plots.
# If you are running this yourself and want interactivity,
# try `%matplotlib widget` instead.
set_matplotlib_formats("svg")
%matplotlib inline
# %matplotlib widget
# Set a consistent plotting style across the notebook using Seaborn.
sns.set_style("darkgrid")
sns.set_context("notebook")
# Make pandas cooperate with pyplot
pd.plotting.register_matplotlib_converters()


C:\Users\EMoses\AppData\Local\Temp\ipykernel_17240\99259732.py:4: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats("svg")


1. Processing and cleaning the bike data
Before getting anywhere with it, we'll need to process the bike data quite a bit. The data comes in CSV files, each of which covers a period of time. Up first, we need to download the data from the TfL website. If you are running this code yourself, here's a script that does that. Be warned though, it's almost seven gigs of data. You can run it repeatedly, and it'll only download data that it doesn't have already.

In [12]:
bikefolder = "data/bikes"

In [13]:
def download_file(datafolder, url, verbosity=0):
    """Download the data from the given URL into the datafolder, unless it's
    already there. Return path to downloaded file.
    """
    datafolder = Path(datafolder)
    datafolder.mkdir(parents=True, exist_ok=True)

    a = urlparse(url)
    filename = Path(os.path.basename(a.path))
    filepath = datafolder / filename
    # Don't redownload if we already have this file.
    if filepath.exists():
        if verbosity > 1:
            print("Already have {}".format(filename))
    else:
        if verbosity > 0:
            print("Downloading {}".format(filename))
        rqst = requests.get(url)
        rqst.raise_for_status()
        with open(filepath, "wb") as f:
            f.write(rqst.content)
    return filepath


In [29]:
# Adjust whether to print progress reports of the downloads.
# verbosity=0 is silence, verbosity=1 reports only when actually doing things,
# verbosity>1 also reports when there's nothing to do.
verbosity = 1

# Most files are individual CSV files, listed in bike_data_urls.txt. Download them.
urlsfile = "data/bikes/bike_data_urls.txt"
with open(urlsfile, "r") as f:
    urls = f.read().splitlines()
# There are a few comments in the file, marked by lines starting with #.
# Filter them out.
urls = [u for u in urls if u[0] != "#"]
for url in urls:
    download_file(bikefolder, url, verbosity)

# The early years come in zips. Download and unzip them.
zipsfolder = Path("data/bikes/bikezips")
bikezipurls = [
    "https://cycling.data.tfl.gov.uk/usage-stats/cyclehireusagestats-2012.zip",
    "https://cycling.data.tfl.gov.uk/usage-stats/cyclehireusagestats-2013.zip",
    "https://cycling.data.tfl.gov.uk/usage-stats/cyclehireusagestats-2014.zip",
    "https://cycling.data.tfl.gov.uk/usage-stats/2015TripDatazip.zip",
    "https://cycling.data.tfl.gov.uk/usage-stats/2016TripDataZip.zip",
]
# A list of CSV files that are already there. Only unzip if some of the files
# in the zip aren't present already.
current_csvs = sorted(os.listdir(bikefolder))
for url in bikezipurls:
    zippath = download_file(zipsfolder, url, verbosity)
    with zipfile.ZipFile(zippath, "r") as z:
        namelist = z.namelist()
        has_been_extracted = any(name not in current_csvs for name in namelist)
        if has_been_extracted:
            if verbosity > 0:
                print("Unzipping {}".format(zippath))
            z.extractall(bikefolder)
        else:
            if verbosity > 1:
                print("{} has already been extracted.".format(zippath))

# Finally, there's an odd one out: One week's data comes in as an .xlsx.
# Download it and use pandas to convert it to csv.
xlsxurl = "https://cycling.data.tfl.gov.uk/usage-stats/49JourneyDataExtract15Mar2017-21Mar2017.xlsx"
xlsxfile = download_file(bikefolder, xlsxurl)
csvfile = xlsxfile.with_suffix(".csv")
if not csvfile.exists():
    if verbosity > 0:
        print("Converting .xlsx to .csv.")
    pd.read_excel(xlsxfile).to_csv(csvfile, date_format="%d/%m/%Y %H:%M:%S")
else:
    if verbosity > 1:
        print("Already have {}".format(csvfile))

The data we have now lists on each line of the CSV file a single bike trip, with starting point and time, end point and time, and things like bike ID number. Here's an example.

In [30]:
example_file  = Path(bikefolder) / Path("47JourneyDataExtract01Mar2017-07Mar2017.csv")
pd.read_csv(example_file, encoding="ISO-8859-2").head()

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name
0,62857677,3780.0,7851,06/03/2017 19:20,43.0,"Crawford Street, Marylebone",06/03/2017 18:17,811,"Westferry Circus, Canary Wharf"
1,62863035,540.0,4089,06/03/2017 22:17,295.0,"Swan Street, The Borough",06/03/2017 22:08,272,"Baylis Road, Waterloo"
2,62775896,600.0,4895,02/03/2017 21:27,295.0,"Swan Street, The Borough",02/03/2017 21:17,197,"Stamford Street, South Bank"
3,62747748,420.0,4347,01/03/2017 21:08,295.0,"Swan Street, The Borough",01/03/2017 21:01,803,"Southwark Street, Bankside"
4,62843939,420.0,3192,06/03/2017 09:28,193.0,"Bankside Mix, Bankside",06/03/2017 09:21,197,"Stamford Street, South Bank"


In [31]:
bikefolder

'data/bikes'

In [32]:
from glob import glob 

# using glob to list all the csv file in the bikefolder filepath
all_csv = glob(bikefolder+str('/*.csv'))
all_csv

['data/bikes\\01aJourneyDataExtract10Jan16-23Jan16.csv',
 'data/bikes\\01bJourneyDataExtract24Jan16-06Feb16.csv',
 'data/bikes\\02aJourneyDataExtract07Feb16-20Feb2016.csv',
 'data/bikes\\02bJourneyDataExtract21Feb16-05Mar2016.csv',
 'data/bikes\\03JourneyDataExtract06Mar2016-31Mar2016.csv',
 'data/bikes\\04JourneyDataExtract01Apr2016-30Apr2016.csv',
 'data/bikes\\05JourneyDataExtract01May2016-17May2016.csv',
 'data/bikes\\06JourneyDataExtract18May2016-24May2016.csv',
 'data/bikes\\07JourneyDataExtract25May2016-31May2016.csv',
 'data/bikes\\08JourneyDataExtract01Jun2016-07Jun2016.csv',
 'data/bikes\\09JourneyDataExtract08Jun2016-14Jun2016.csv',
 'data/bikes\\1. Journey Data Extract 01Jan-05Jan13.csv',
 'data/bikes\\1. Journey Data Extract 04Jan-31Jan 12.csv',
 'data/bikes\\1. Journey Data Extract 05Jan14-02Feb14.csv',
 'data/bikes\\10. Journey Data Extract 18Aug-13Sep13.csv',
 'data/bikes\\10. Journey Data Extract 21Aug-22 Aug12.csv',
 'data/bikes\\10a Journey Data Extract 20Sep15-03Oct

In [33]:
# creating a list of csv files that contain '2019' and '2022' respectively
csv_2019 = [item for item in all_csv if '2019' in item]
csv_2022 = [item for item in all_csv if '2022' in item]

['data/bikes\\298JourneyDataExtract29Dec2021-04Jan2022.csv',
 'data/bikes\\299JourneyDataExtract05Jan2022-11Jan2022.csv',
 'data/bikes\\300JourneyDataExtract12Jan2022-18Jan2022.csv',
 'data/bikes\\301JourneyDataExtract19Jan2022-25Jan2022.csv',
 'data/bikes\\302JourneyDataExtract26Jan2022-01Feb2022.csv',
 'data/bikes\\303JourneyDataExtract02Feb2022-08Feb2022.csv',
 'data/bikes\\304JourneyDataExtract09Feb2022-15Feb2022.csv',
 'data/bikes\\305JourneyDataExtract16Feb2022-22Feb2022.csv',
 'data/bikes\\306JourneyDataExtract23Feb2022-01Mar2022.csv',
 'data/bikes\\307JourneyDataExtract02Mar2022-08Mar2022.csv',
 'data/bikes\\308JourneyDataExtract09Mar2022-15Mar2022.csv',
 'data/bikes\\309JourneyDataExtract16Mar2022-22Mar2022.csv',
 'data/bikes\\310JourneyDataExtract23Mar2022-29Mar2022.csv',
 'data/bikes\\311JourneyDataExtract30Mar2022-05Apr2022.csv',
 'data/bikes\\312JourneyDataExtract06Apr2022-12Apr2022.csv',
 'data/bikes\\313JourneyDataExtract13Apr2022-19Apr2022.csv',
 'data/bikes\\314Journey

In [34]:
# using list comprehension that reads each csv file from the list and gnerators a sequence of dataframes
dfs = (pd.read_csv(csv) for csv in csv_2019)

# concatenate csvs them into a single DataFrame using pd.concat()
# ignore_index=True parameter resets the index of the resulting DataFrame, so that it is a continuous sequence of integers.
data_2019 = pd.concat(dfs, ignore_index=True)

In [40]:
print(csv_2022)

['data/bikes\\298JourneyDataExtract29Dec2021-04Jan2022.csv', 'data/bikes\\299JourneyDataExtract05Jan2022-11Jan2022.csv', 'data/bikes\\300JourneyDataExtract12Jan2022-18Jan2022.csv', 'data/bikes\\301JourneyDataExtract19Jan2022-25Jan2022.csv', 'data/bikes\\302JourneyDataExtract26Jan2022-01Feb2022.csv', 'data/bikes\\303JourneyDataExtract02Feb2022-08Feb2022.csv', 'data/bikes\\304JourneyDataExtract09Feb2022-15Feb2022.csv', 'data/bikes\\305JourneyDataExtract16Feb2022-22Feb2022.csv', 'data/bikes\\306JourneyDataExtract23Feb2022-01Mar2022.csv', 'data/bikes\\307JourneyDataExtract02Mar2022-08Mar2022.csv', 'data/bikes\\308JourneyDataExtract09Mar2022-15Mar2022.csv', 'data/bikes\\309JourneyDataExtract16Mar2022-22Mar2022.csv', 'data/bikes\\310JourneyDataExtract23Mar2022-29Mar2022.csv', 'data/bikes\\311JourneyDataExtract30Mar2022-05Apr2022.csv', 'data/bikes\\312JourneyDataExtract06Apr2022-12Apr2022.csv', 'data/bikes\\313JourneyDataExtract13Apr2022-19Apr2022.csv', 'data/bikes\\314JourneyDataExtract20Apr

In [37]:
# doing the same for the 2022 data
dfs_2022 = (pd.read_csv(csv) for csv in csv_2022)

data_2022 = pd.concat(dfs_2022, ignore_index=True)

ParserError: Error tokenizing data. C error: EOF inside string starting at row 8078

In [23]:
print(data_2019.shape)
data_2019.head()
print(data_2022.shape)
data_2022.head()

(10388411, 9)


NameError: name 'data_2022' is not defined

In [ ]:
## Add some extra variables to the dataset for use later in filtering

import datetime

## Feeding a specififed date format speeds up the pd.to_datetime function immeasurably, especially over large datasets
## e.g. http://stackoverflow.com/questions/32034689/why-is-pandas-to-datetime-slow-for-non-standard-time-format-such-as-2014-12-31

format = "%d/%m/%Y %H:%M"

## Some routes had dates with a seconds component, whereas some didn't - the below code cuts these seconds off
data_2019['Start Date']= data_2019['Start Date'].str[:16]

data_2019['Start Date Converted']= pd.to_datetime(data_2019['Start Date'], format=format).dt.date

data_2019['Hours']= pd.to_datetime(data_2019['Start Date'], format=format).dt.hour

data_2019['Day']= pd.to_datetime(data_2019['Start Date'], format=format).dt.weekday

data_2019.head()


,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name,Start Date Converted,Hours,Day
0,83252102,720,2077,31/12/2018 19:05,272,"Baylis Road, Waterloo",31/12/2018 18:53,94,"Bricklayers Arms, Borough",2018-12-31,18,0
1,83195883,120,10781,27/12/2018 19:47,93,"Cloudesley Road, Angel",27/12/2018 19:45,339,"Risinghill Street, Angel",2018-12-27,19,3
2,83196070,120,2977,27/12/2018 20:11,339,"Risinghill Street, Angel",27/12/2018 20:09,234,"Liverpool Road (N1 Centre), Angel",2018-12-27,20,3
3,83197932,660,10802,28/12/2018 07:35,282,"Royal London Hospital, Whitechapel",28/12/2018 07:24,698,"Shoreditch Court, Haggerston",2018-12-28,7,4
4,83176351,1380,15749,26/12/2018 11:55,785,"Aquatic Centre, Queen Elizabeth Olympic Park",26/12/2018 11:32,783,"Monier Road, Hackney Wick",2018-12-26,11,2


In [ ]:
data_2022.shape

#data_2022.sort_values(by='End date', ascending=True)
data_2022.sort_values(by='Bike model', ascending=False)


,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name,Number,Start date,Start station number,Start station,End date,End station number,End station,Bike number,Bike model,Total duration,Total duration (ms)
10553852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126646858.0,2022-11-09 10:24,22180,"Cheapside, Bank",2022-11-09 10:33,1151,"Whitehall Place, Strand",60399.0,PBSC_EBIKE,9m 34s,574742.0
9961990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126117045.0,2022-10-19 21:42,300012,"Irene Road, Parsons Green",2022-10-19 21:48,300058,"The Vale, Chelsea",60402.0,PBSC_EBIKE,6m 3s,363321.0
10109385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126367091.0,2022-10-28 18:30,3467,"Dock Street, Wapping",2022-10-28 18:56,200129,"Charlotte Terrace, Angel",60130.0,PBSC_EBIKE,26m 0s,1560647.0
10109380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126367086.0,2022-10-28 18:30,973,"Bethnal Green Road, Shoreditch",2022-10-28 18:57,300009,"World's End Place, West Chelsea",60150.0,PBSC_EBIKE,27m 14s,1634782.0
10342610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126498212.0,2022-11-02 16:14,1186,"Belvedere Road 1, South Bank",2022-11-02 16:27,2667,"Tower Wharf, Bermondsey",60272.0,PBSC_EBIKE,13m 20s,800924.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8930282,125044170.0,3840.0,21738.0,11/09/2022 09:45,201.0,"Dorset Square, Marylebone",11/09/2022 08:41,514.0,"Portman Square, Marylebone",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8930283,125102716.0,360.0,10886.0,11/09/2022 10:19,5.0,"Sedding Street, Sloane Square",11/09/2022 10:13,826.0,"Allington Street, Victoria",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8930284,125102718.0,540.0,8719.0,11/09/2022 10:53,270.0,"Kennington Lane Rail Bridge, Vauxhall",11/09/2022 10:44,826.0,"Allington Street, Victoria",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8930285,125044249.0,6000.0,8881.0,11/09/2022 14:13,757.0,"Harcourt Terrace, West Brompton",11/09/2022 12:33,757.0,"Harcourt Terrace, West Brompton",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# In September 2022 the column names change slightly and additional clumns have been added
# for example the 'Bike model' column has been added (classic or PBSC_EBIKE)

# Let's clean this up and get all the data into single columns
# transfering values from one pandas column to another pandas column only for null rows

#creating a copy of the orginal data
data_2022_clean = data_2022.copy()




In [ ]:
data_2022_clean.loc[data_2022_clean['Rental Id'].isnull(), 'Rental Id'] = data_2022_clean['Number']
# converting from milliseconds to seconds, multipyling by 1000 
data_2022_clean.loc[data_2022_clean['Duration'].isnull(), 'Duration'] = data_2022_clean['Total duration (ms)'] / 1000
data_2022_clean.loc[data_2022_clean['Bike Id'].isnull(), 'Bike Id'] = data_2022_clean['Bike number']
data_2022_clean.loc[data_2022_clean['End Date'].isnull(), 'End Date'] = data_2022_clean['End date']
#data_2022_clean.loc[data_2022_clean['EndStation Id'].isnull(), 'EndStation Id'] = data_2022_clean['End station number']
data_2022_clean.loc[data_2022_clean['EndStation Name'].isnull(), 'EndStation Name'] = data_2022_clean['End station']
data_2022_clean.loc[data_2022_clean['Start Date'].isnull(), 'Start Date'] = data_2022_clean['Start date']

data_2022_clean.sort_values(by='Bike model', ascending=False)

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name,Number,Start date,Start station number,Start station,End date,End station number,End station,Bike number,Bike model,Total duration,Total duration (ms)
10553852,126646858.0,574.742,60399.0,2022-11-09 10:33,NaN,"Whitehall Place, Strand",2022-11-09 10:24,NaN,NaN,126646858.0,2022-11-09 10:24,22180,"Cheapside, Bank",2022-11-09 10:33,1151,"Whitehall Place, Strand",60399.0,PBSC_EBIKE,9m 34s,574742.0
9961990,126117045.0,363.321,60402.0,2022-10-19 21:48,NaN,"The Vale, Chelsea",2022-10-19 21:42,NaN,NaN,126117045.0,2022-10-19 21:42,300012,"Irene Road, Parsons Green",2022-10-19 21:48,300058,"The Vale, Chelsea",60402.0,PBSC_EBIKE,6m 3s,363321.0
10109385,126367091.0,1560.647,60130.0,2022-10-28 18:56,NaN,"Charlotte Terrace, Angel",2022-10-28 18:30,NaN,NaN,126367091.0,2022-10-28 18:30,3467,"Dock Street, Wapping",2022-10-28 18:56,200129,"Charlotte Terrace, Angel",60130.0,PBSC_EBIKE,26m 0s,1560647.0
10109380,126367086.0,1634.782,60150.0,2022-10-28 18:57,NaN,"World's End Place, West Chelsea",2022-10-28 18:30,NaN,NaN,126367086.0,2022-10-28 18:30,973,"Bethnal Green Road, Shoreditch",2022-10-28 18:57,300009,"World's End Place, West Chelsea",60150.0,PBSC_EBIKE,27m 14s,1634782.0
10342610,126498212.0,800.924,60272.0,2022-11-02 16:27,NaN,"Tower Wharf, Bermondsey",2022-11-02 16:14,NaN,NaN,126498212.0,2022-11-02 16:14,1186,"Belvedere Road 1, South Bank",2022-11-02 16:27,2667,"Tower Wharf, Bermondsey",60272.0,PBSC_EBIKE,13m 20s,800924.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8930282,125044170.0,3840.000,21738.0,11/09/2022 09:45,201.0,"Dorset Square, Marylebone",11/09/2022 08:41,514.0,"Portman Square, Marylebone",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8930283,125102716.0,360.000,10886.0,11/09/2022 10:19,5.0,"Sedding Street, Sloane Square",11/09/2022 10:13,826.0,"Allington Street, Victoria",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8930284,125102718.0,540.000,8719.0,11/09/2022 10:53,270.0,"Kennington Lane Rail Bridge, Vauxhall",11/09/2022 10:44,826.0,"Allington Street, Victoria",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8930285,125044249.0,6000.000,8881.0,11/09/2022 14:13,757.0,"Harcourt Terrace, West Brompton",11/09/2022 12:33,757.0,"Harcourt Terrace, West Brompton",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data_2022_clean.dtypes

Rental Id               float64
Duration                float64
Bike Id                 float64
End Date                 object
EndStation Id           float64
EndStation Name          object
Start Date               object
StartStation Id         float64
StartStation Name        object
Number                  float64
Start date               object
Start station number     object
Start station            object
End date                 object
End station number       object
End station              object
Bike number             float64
Bike model               object
Total duration           object
Total duration (ms)     float64
dtype: object

In [ ]:
# cleaning the 2022 data following the same process
data_2022['Start Date']= data_2022['Start Date'].str[:16]

data_2022['Start Date Converted']= pd.to_datetime(data_2022['Start Date'], format=format).dt.date

data_2022['Hours']= pd.to_datetime(data_2022['Start Date'], format=format).dt.hour

data_2022['Day']= pd.to_datetime(data_2022['Start Date'], format=format).dt.weekday

data_2022.head()


In [ ]:
# Convert the date column to datetime format
data_2019['Start Date Converted'] = pd.to_datetime(data_2019['Start Date Converted'])
data_2022['Start Date Converted'] = pd.to_datetime(data_2022['Start Date Converted'])

KeyError: 'Start Date Converted'

In [ ]:
# Filter the dataframe to only include data from 2019
# Remember, one of the merged csv files contain data between 26/12/2018 to 01/01/2019 - filtering will remove the 2018 data 
bike_data_2019 = data_2019[data_2019['Start Date Converted'].dt.year == 2019]
print(bike_data_2019.shape)

# 2022 filtering data
bike_data_2022 = data_2022[data_2022['Start Date Converted'].dt.year == 2022]
print(bike_data_2022.shape)

(10310063, 12)
(8874274, 23)


---
---

### Exploratory Data Analysis 
---

In [ ]:
# counting by start location in 2019
print(bike_data_2019['StartStation Name'].value_counts())
# counting by end location in 2019
print(bike_data_2019['EndStation Name'].value_counts())


Belgrove Street , King's Cross          84578
Waterloo Station 3, Waterloo            77998
Hyde Park Corner, Hyde Park             71834
Waterloo Station 1, Waterloo            65950
Albert Gate, Hyde Park                  55133
                                        ...  
Warwick Row, Westminster                  373
Blackfriars Station, St. Paul's           352
St Mary's Hospital, Westminster           119
Pop Up Dock 1                              57
Thessaly Road North, Wandsworth Road        7
Name: StartStation Name, Length: 801, dtype: int64
Belgrove Street , King's Cross          83725
Waterloo Station 3, Waterloo            74818
Hyde Park Corner, Hyde Park             72300
Hop Exchange, The Borough               63806
Waterloo Station 1, Waterloo            62442
                                        ...  
Warwick Row, Westminster                  387
Blackfriars Station, St. Paul's           336
St Mary's Hospital, Westminster           132
Pop Up Dock 1                

In [ ]:
# counting by start location in 2022
print(bike_data_2022['StartStation Name'].value_counts())
# counting by end location in 2022
print(bike_data_2022['EndStation Name'].value_counts())

Hyde Park Corner, Hyde Park        73427
Albert Gate, Hyde Park             45748
Wellington Arch, Hyde Park         41710
Belgrove Street , King's Cross     40859
Hop Exchange, The Borough          39684
                                   ...  
Lansdown Drive, Hackney Central      630
Portugal Street, Holborn             512
Rossmore Road, Marylebone            511
Brandon Street, Walworth             276
Cartier Circle, Canary Wharf          75
Name: StartStation Name, Length: 803, dtype: int64
Hyde Park Corner, Hyde Park        73316
Hop Exchange, The Borough          51630
Albert Gate, Hyde Park             45611
Wellington Arch, Hyde Park         41703
Belgrove Street , King's Cross     40905
                                   ...  
Lansdown Drive, Hackney Central      649
Portugal Street, Holborn             491
Rossmore Road, Marylebone            460
Brandon Street, Walworth             280
Cartier Circle, Canary Wharf          58
Name: EndStation Name, Length: 803, dtype: int6

In [ ]:
# counting the number of rows in teh dataframes by month using the groupby() function and the dt.month attribute of the Pandas DateTime object

counts_2019 = bike_data_2019.groupby(bike_data_2019['Start Date Converted'].dt.month).count()
counts_2022 = bike_data_2022.groupby(bike_data_2022['Start Date Converted'].dt.month).count()

print(counts_2022)

                      Rental Id  Duration  Bike Id  End Date  EndStation Id  \
Start Date Converted                                                          
1                        741875    741875   741875    741875         741875   
2                        743581    743581   743581    743581         743581   
3                       1047369   1047369  1047369   1047369        1047369   
4                       1022467   1022467  1022467   1022467        1022467   
5                       1190590   1190590  1190590   1190590        1190590   
6                       1268701   1268701  1268701   1268701        1268701   
7                       1303376   1303376  1303376   1303376         991232   
8                       1249038   1249038  1249038   1249038        1249038   
9                        307277    307277   307277    307277         307277   

                      EndStation Name  Start Date  StartStation Id  \
Start Date Converted                                        

# Getting the bike station locations
##### TfL have a live "cycle hire updates" feed which lists information for each cycle hire station, updated once every minute or so. I don't utilise this live data - instead I just take the name, ID, lat/lon, and capacity for each bike station.

Code adopted from https://github.com/charlie1347/TfL_bikes/blob/master/TfL%20Bikes.ipynb


In [ ]:
import requests
from xml.etree import ElementTree as ET
import pandas as pd

site = "https://tfl.gov.uk/tfl/syndication/feeds/cycle-hire/livecyclehireupdates.xml"

response = requests.get(site)
root = ET.fromstring(response.content)

id_list = [int(root[i][0].text) for i in range(0, len(root))]
name_list = [root[i][1].text for i in range(0, len(root))]
lat_list = [float(root[i][3].text) for i in range(0, len(root))]
lon_list = [float(root[i][4].text) for i in range(0, len(root))]
capacity_list = [int(root[i][12].text) for i in range(0, len(root))]

all_locs = pd.DataFrame(list(zip(name_list, id_list, lat_list, 
                                 lon_list, capacity_list)), columns = ["name","id","lat","lon","capacity"])

all_locs.to_csv('output/bike_point_locations_saved.csv', header=True, index=None)

print(all_locs.shape)


locations = all_locs.copy()

locations.head()

(799, 5)


,name,id,lat,lon,capacity
0,"River Street , Clerkenwell",1,51.529163,-0.109971,0
1,"Phillimore Gardens, Kensington",2,51.499607,-0.197574,0
2,"Christopher Street, Liverpool Street",3,51.521284,-0.084606,0
3,"St. Chad's Street, King's Cross",4,51.530059,-0.120974,0
4,"Sedding Street, Sloane Square",5,51.493130,-0.156876,0


### Creating an interative map of TfL bike docking stations

In [ ]:
import folium 
# creating a folium map 

m = folium.Map(
    location=[51.5074, -0.1272], # start location as lat and lon
    tiles="CartoDB dark_matter", # adding a dark basemap
    zoom_start=12, # level of zoom
    prefer_canvas=True, # useful for changing the base map
)

In [ ]:
# iterate through every row of the dataframe using the iterrows() function
for index, val in locations.iterrows():
    folium.CircleMarker(
        location=[val["lat"], val["lon"]],
        # styling the circles with different parameters 
        radius=5,
        popup= val["name"],
        color='#FF0000',
        fill=True,
        fill_color='#3186cc'
    # adding all circles to the map
    ).add_to(m)


In [ ]:
m 

### Merging all bike data from 2019 and docking locations into one dataframe